In [1]:
import pandas as pd
import hashlib

In [2]:
data_path = "../../../database"

df = pd.read_csv(f"{data_path}/silver/arquivo_anonimizado_v2.csv")

In [3]:
colunas_curso = [
    "sk_d_curso",
    "cod_curso_ingresso",
    "curso_ingresso_ufrj",
    "cod_curso_atual",
    "curso_atual",
]
d_curso = df[colunas_curso[1:]]
d_curso = d_curso.drop_duplicates().reset_index(drop=True)

In [4]:
d_curso["sk_d_curso"] = d_curso.apply(
    lambda x: hashlib.sha256(
        str(x["cod_curso_ingresso"] + x["cod_curso_atual"]).encode()
    ).hexdigest(),
    axis=1,
)
d_curso = d_curso.reindex(columns=colunas_curso)

In [5]:
d_curso.to_csv(f"{data_path}/gold/d_curso.csv", index=False)